In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, "../src/features")
import data_cleaning

# allows all columns to be displayed
pd.set_option('display.max_columns', None)

OFFER_DATA_DIR = "../data/offer_acceptance_offers.csv"
ORDER_DATA_DIR = "../data/offer_acceptance_orders.csv"

offers = pd.read_csv(OFFER_DATA_DIR, low_memory=False)[["CARRIER_ID", "REFERENCE_NUMBER", "CREATED_ON_HQ", "RATE_USD", "OFFER_TYPE", "LOAD_DELIVERED_FROM_OFFER"]]
orders = pd.read_csv(ORDER_DATA_DIR, low_memory=False)[["REFERENCE_NUMBER", "ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST", "ORIGIN_3DIGIT_ZIP", "DESTINATION_3DIGIT_ZIP", "APPROXIMATE_DRIVING_ROUTE_MILEAGE", "PALLETIZED_LINEAR_FEET"]]

offers = data_cleaning.change_to_date(offers, ["CREATED_ON_HQ"])
orders = data_cleaning.change_to_date(orders, ["ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST"])

orders = data_cleaning.parse_zipcode(orders)

offers = data_cleaning.flatten_ref_num(offers)
orders = data_cleaning.flatten_ref_num(orders)

merged = data_cleaning.join_offers_orders(offers, orders, how="inner")

merged = data_cleaning.get_remaining_time(merged)

merged = data_cleaning.during_business_hours(merged)

pooled = data_cleaning.get_prorated_rate(merged)

merged = data_cleaning.impute_mileage(merged)

merged = data_cleaning.get_business_hours(merged)

merged = data_cleaning.get_off_business_hours(merged)

merged = data_cleaning.get_weekday(merged)

In [5]:
merged = data_cleaning.get_remaining_time_in_seconds(merged, "REMAINIG_TIME")

In [7]:
 # filter for delivered offers
merged = merged[merged["LOAD_DELIVERED_FROM_OFFER"]].reset_index(drop=True)

 # filter for quote offers
merged = merged[merged["OFFER_TYPE"] == "quote"]

# drop unnecessary columns
merged = merged.drop(["CARRIER_ID", "REFERENCE_NUMBER", "LOAD_DELIVERED_FROM_OFFER", "CREATED_ON_HQ", "ORIGIN_3DIGIT_ZIP", "DESTINATION_3DIGIT_ZIP"], axis=1)

In [8]:
merged

,RATE_USD,OFFER_TYPE,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,ORIGIN_CITY,DESTINATION_CITY,REMAINIG_TIME,BUSINESS_HOURS,BUSINESS_HOURS_ORDER_PICKUP,OFF_BUSINESS_HOURS_ORDER_PICKUP,WEEKDAY_NUM
2,1096.5000,quote,2022-07-21 06:17:07,2022-07-28 13:00:00,318.0,14,Rancho Cucamonga CA,Glendale AZ,470721.0,False,55.000000,108.0,3
3,1806.0000,quote,2022-07-26 10:22:11,2022-07-28 14:00:00,368.0,31,Whittier CA,Phoenix AZ,89198.0,True,23.630278,28.0,3
7,3870.0000,quote,2022-09-08 13:19:35,2022-09-09 13:00:00,2659.0,15,Whittier CA,West Palm Beach FL,15004.0,True,9.673611,14.0,4
18,4902.0000,quote,2022-03-08 09:01:18,2022-03-11 16:00:00,2186.0,32,San Bernardino CA,Gainesville GA,83617.0,True,36.978333,42.0,4
21,1935.0000,quote,2022-03-04 10:58:02,2022-03-08 11:00:00,966.0,21,Dallas TX,Chicago IL,18432.0,True,20.032778,66.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165350,967.5000,quote,2022-06-28 11:20:10,2022-06-29 14:00:00,408.0,52,Oakland CA,Whittier CA,95950.0,True,12.663889,14.0,2
165351,1032.0000,quote,2020-10-22 05:30:25,2020-10-23 11:30:00,413.0,52,Atlanta GA,Jackson MS,107099.0,True,13.500000,14.0,4
165352,1290.0000,quote,2021-08-17 09:25:59,2021-08-18 13:00:00,397.0,22,Dallas TX,Hutchinson KS,90673.0,True,13.566944,14.0,2
165353,6450.0000,quote,2021-06-29 05:18:36,2021-06-30 16:00:00,1080.0,29,San Diego CA,Longmont CO,28273.0,True,18.000000,14.0,2


In [9]:
merged.columns

Index(['RATE_USD', 'OFFER_TYPE', 'ORDER_DATETIME_PST', 'PICKUP_DEADLINE_PST',
       'APPROXIMATE_DRIVING_ROUTE_MILEAGE', 'PALLETIZED_LINEAR_FEET',
       'ORIGIN_CITY', 'DESTINATION_CITY', 'REMAINIG_TIME', 'BUSINESS_HOURS',
       'BUSINESS_HOURS_ORDER_PICKUP', 'OFF_BUSINESS_HOURS_ORDER_PICKUP',
       'WEEKDAY_NUM'],
      dtype='object')

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [20]:
MAX_CATEGORIES = 30

df = merged.drop(["ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST", "OFFER_TYPE"], axis=1)

df_X = df.drop("RATE_USD", axis=1)
df_y = df["RATE_USD"]

df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=42)

num_feat = ["APPROXIMATE_DRIVING_ROUTE_MILEAGE", "PALLETIZED_LINEAR_FEET", "REMAINIG_TIME", 'BUSINESS_HOURS_ORDER_PICKUP', 'OFF_BUSINESS_HOURS_ORDER_PICKUP']
num_transformer = Pipeline(steps=[
    ('scaler', MaxAbsScaler())
])

cat_feat = ['ORIGIN_CITY', 'DESTINATION_CITY', 'BUSINESS_HOURS', 'WEEKDAY_NUM']
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(max_categories = 30, handle_unknown = 'ignore'))     # output from Ordinal becomes input to OneHot
])

# preprocessing pipeline (put them together)
preproc = ColumnTransformer(
    transformers=[
        ("numerical", num_transformer, num_feat),
        ("categorization", cat_transformer, cat_feat)
    ])

In [21]:
regressors = []

lasso_model = Lasso()
regressors.append(lasso_model)

gidge_model = Ridge()
regressors.append(gidge_model)

for reg in regressors:
    pl = Pipeline(steps=[('preprocessor', preproc), (str(reg), reg)])
    
    pl.fit(df_X_train, df_y_train)
    y_preds = pl.predict(df_X_test)

    mse = mean_squared_error(df_y_test, y_preds)
    print("MSE of %s is %s"%(reg, mse))

MSE of Lasso() is 1312021.4322382417
MSE of Ridge() is 1299863.0447060326


In [30]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Ridge())])

parameters = {
    "regressor__alpha":[0.0001, 0.001, 0.01,0.1,1,10,100,1000]
}


# initialize
grid_pipeline = GridSearchCV(pl,parameters, cv = 8)


grid_pipeline.fit(df_X_train,df_y_train)

grid_pipeline.best_params_

{'regressor__alpha': 0.001}

In [28]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Ridge(alpha = 0.001))])

pl.fit(df_X_train, df_y_train)
y_preds = pl.predict(df_X_test)

mse = mean_squared_error(df_y_test, y_preds)
mse

1299673.8601404575